# Netflix - Sistema de Recomendação por Similaridade de Conteúdo

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

## Carga do Dataset

O dataset contém informações sobre filmes e séries disponíveis na Netflix, incluindo título, gênero, país, data de adição, classificação etária, duração, tipo (filme ou série) e descrição.

Descrição de cada coluna:
- show_id: Identificador único do show.
- type: Tipo do show (Filme ou Série).
- title: Título do show.
- director: Diretor do show.
- cast: Elenco do show.
- country: País de origem do show.
- date_added: Data em que o show foi adicionado à Netflix.
- release_year: Ano de lançamento do show.
- rating: Classificação etária do show.
- duration: Duração do show (em minutos para filmes, número de temporadas para séries).
- listed_in: Gêneros do show.

In [27]:
# Carrega o dataset
data = pd.read_csv('../data/netflix_titles.csv')

## Pré-processamento básico

Realiza etapas fundamentais de limpeza e preparação dos dados textuais antes da vetorização com TF-IDF.

- Verificam e preenchem valores nulos (NaN) nas colunas 'cast', 'country' e 'listed_in'.
- Substituem valores ausentes por strings vazias ('').
- Isso é importante porque o modelo de vetorização textual (TfidfVectorizer) não aceita valores nulos — ele espera apenas strings.

In [28]:
# Preenche valores nulos com string vazia (para campos textuais)
data['cast'] = data['cast'].fillna('')
data['country'] = data['country'].fillna('')
data['listed_in'] = data['listed_in'].fillna('')

# Junta as colunas relevantes em uma só para vetorizacao
data['conteudo'] = data['cast'] + ' ' + data['country'] + ' ' + data['listed_in']

### Vetorização TF-IDF

Transforma os dados textuais do catálogo da Netflix em representações vetoriais numéricas, permitindo comparar quão semelhantes são dois títulos com base em seu conteúdo.

In [29]:
#Transforma textos em vetores numéricos usando a técnica TF-IDF (Term Frequency-Inverse Document Frequency).
#Remove palavras comuns do inglês (como "the", "and", "is") que não agregam significado à análise.
vectorizer = TfidfVectorizer(stop_words='english')

#Aprende o vocabulário dos textos na coluna 'conteudo' e transforma cada linha (título) em um vetor TF-IDF correspondente
tfidf_matrix = vectorizer.fit_transform(data['conteudo'])

# Calcula a similaridade do cosseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Função de recomendação por título

Essa função retorna os títulos mais semelhantes ao fornecido, com base na similaridade do cosseno entre os vetores TF-IDF dos conteúdos.

- Cosine_sim[idx] retorna um vetor com a similaridade do título com todos os outros.
- Enumerate associa cada similaridade ao respectivo índice do título no dataset.
- A lista é ordenada da maior para a menor similaridade.
- O primeiro resultado é o próprio título, então é removido com sim_scores[1:].

In [30]:
def get_recomendacoes(titulo, top_n=5):
    # Verifica se o título existe
    if titulo.lower() not in data['title'].str.lower().values:
        return ["Título não encontrado"]

    # Obtém o índice da linha correspondente ao título na base de dados.
    idx = data[data['title'].str.lower() == titulo.lower()].index[0]

    # Similaridades com todos os outros títulos
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # ignora o próprio título

    # Índices dos recomendados
    indices = [i[0] for i in sim_scores]

    return data[['title', 'type', 'release_year', 'rating', 'listed_in']].iloc[indices].to_dict(orient='records')

### Função de recomendação por campos

Essa função gera recomendações de títulos da Netflix sem depender do título original. Em vez disso, utiliza campos como elenco (cast), país (country) e categorias (listed_in) para encontrar conteúdos semelhantes com base no significado textual.

In [31]:
def recomendar_por_campos(fields, top_n=5):
    # Extrai os campos cast, country e listed_in do dicionário fields enviado pelo usuário.
    # Usa .get() com valor padrão vazio ('') para evitar erros se alguma chave estiver ausente.
    campos = [
        fields.get('cast', ''),
        fields.get('country', ''),
        fields.get('listed_in', '')
    ]

    # Concatena todos os campos em uma única string (imitando o campo conteudo gerado durante o pré-processamento).
    conteudo_novo = ' '.join(campos)

    #Transforma a string criada (conteudo_novo) em um vetor TF-IDF usando o vectorizer já treinado
    #Isso permite comparar semanticamente com o dataset completo.
    vetor = vectorizer.transform([conteudo_novo])

    #Calcula a similaridade do cosseno entre o vetor da entrada e todos os vetores do dataset original (tfidf_matrix).
    #O resultado é uma lista com scores de similaridade.
    sim_scores = cosine_similarity(vetor, tfidf_matrix).flatten()

    #Ordena os índices dos maiores scores (conteúdos mais parecidos).
    #Seleciona os top_n resultados
    indices = np.argsort(sim_scores)[::-1][:top_n]

    return data[['title', 'type', 'release_year']].iloc[indices].to_dict(orient='records')


### Exemplo de uso
Este trecho executa chamadas manuais às duas funções principais de recomendação para verificar o funcionamento do sistema e observar os resultados diretamente

In [32]:
# Chama a função get_recomendacoes usando o título "Naruto" como base.
# Solicita os 5 conteúdos mais similares, com base na similaridade do cosseno entre os vetores TF-IDF.
exemplo = get_recomendacoes('naruto', top_n=5)
for item in exemplo:
    print(item)

# Chama a função recomendar_por_campos com campos específicos
# Busca conteúdos que correspondam aos critérios fornecidos (país, ano de lançamento, classificação, categoria e duração).
for item in recomendar_por_campos({'country': 'Japan', 'release_year': '2006', 'rating': 'TV-14', 'listed_in': 'Anime', 'duration': '9'}, top_n=5):
    print(item)

{'title': 'Naruto Shippûden the Movie: Bonds', 'type': 'Movie', 'release_year': 2008, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippuden the Movie: Blood Prison', 'type': 'Movie', 'release_year': 2011, 'rating': 'TV-14', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippuden : Blood Prison', 'type': 'Movie', 'release_year': 2011, 'rating': 'TV-14', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto the Movie: Ninja Clash in the Land of Snow', 'type': 'Movie', 'release_year': 2004, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippûden the Movie: The Will of Fire', 'type': 'Movie', 'release_year': 2009, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Pop Team Epic', 'type': 'TV Show', 'release_year': 2018}
{'title': 

### Exportação dos objetos treinados com pickle
Exportar para disco os principais componentes do sistema de recomendação, permitindo que sejam reutilizados em uma aplicação externa (como uma API Flask) sem necessidade de recalcular tudo novamente.

In [33]:
with open('../models/dados_netflix.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('../models/cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

with open('../models/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)